In [26]:
import numpy as np
import matplotlib.pyplot as plt

class CuatroEnRaya:

    def __init__(self, filas=6, columnas=7): #Iniciar el tablero con las dimensiones dadas
        self.filas = filas
        self.columnas = columnas
        self.tablero = np.zeros((filas, columnas), dtype=int)

    def reiniciar(self): #Reiniciar el tablero a su estado inicial
        self.tablero = np.zeros((self.filas, self.columnas), dtype=int)

    def realizar_movimiento(self, columna, jugador): #Movimientos de los jugadores
        for fila in range(self.filas-1, -1, -1):  #Empieza desde la última fila
            if self.tablero[fila, columna] == 0:
                self.tablero[fila, columna] = jugador
                return True
        return False

    def verificar_victoria(self, jugador): #Verifica si el jugador formo cuatro fichas en alguna direccion

        # Revisar filas
        for fila in range(self.filas):
            for col in range(self.columnas - 3):
                if all(self.tablero[fila, col + i] == jugador for i in range(4)):
                    return True

        # Revisar columnas
        for col in range(self.columnas):
            for fila in range(self.filas - 3):
                if all(self.tablero[fila + i, col] == jugador for i in range(4)):
                    return True

        # Revisar diagonales derecha
        for fila in range(self.filas - 3):
            for col in range(self.columnas - 3):
                if all(self.tablero[fila + i, col + i] == jugador for i in range(4)):
                    return True

        # Revisar diagonales izquierda
        for fila in range(3, self.filas):
            for col in range(self.columnas - 3):
                if all(self.tablero[fila - i, col + i] == jugador for i in range(4)):
                    return True

        return False

In [28]:
# Prueba
juego = CuatroEnRaya()
movimientos = [
    (3, 1),  # Jugador 1 juega en columna 3
    (4, 2),  # Jugador 2 juega en columna 4
    (3, 1),  # Jugador 1 juega en columna 3
    (4, 2),  # Jugador 2 juega en columna 4
    (3, 1),  # Jugador 1 juega en columna 3
    (4, 2),  # Jugador 2 juega en columna 4
    (3, 1)   # Jugador 1 juega en columna 3
]

for columna, jugador in movimientos:
    if juego.realizar_movimiento(columna, jugador):
        if juego.verificar_victoria(jugador):
            print(f"¡El jugador {jugador} ha ganado!")
            break

print(juego.tablero)  # Muestra el estado final del tablero

¡El jugador 1 ha ganado!
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 2 0 0]
 [0 0 0 1 2 0 0]
 [0 0 0 1 2 0 0]]


In [31]:
import numpy as np

class AgenteQlearning:

    def __init__(self, filas=6, columnas=7, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.filas = filas
        self.columnas = columnas
        self.alpha = alpha  # Tasa de aprendizaje
        self.gamma = gamma  # Factor de descuento (cuánto valora futuras recompensas)
        self.epsilon = epsilon  # Probabilidad de exploración
        self.q_tabla = {}  # Diccionario que almacena los valores Q

    def obtener_estado(self, tablero):
        # Convierte el tablero en una representación única en formato de cadena
        return str(tablero.flatten())

    def elegir_accion(self, tablero):
        # Selecciona una accion basada en exploración o explotacion segun la tabla Q
        estado = self.obtener_estado(tablero)
        if np.random.rand() < self.epsilon:  # Explora
            return np.random.choice(range(self.columnas))
        else:  # Explota
            return max(range(self.columnas), key=lambda c: self.q_tabla.get((estado, c), 0))

    def actualizar_q(self, estado, accion, recompensa, estado_siguiente):
        # Actualiza el valor Q
        estado_q = self.q_tabla.get((estado, accion), 0)
        mejor_q_siguiente = max([self.q_tabla.get((estado_siguiente, a), 0) for a in range(self.columnas)], default=0)
        self.q_tabla[(estado, accion)] = estado_q + self.alpha * (recompensa + self.gamma * mejor_q_siguiente - estado_q)

# Prueba inicial del agente
agente = AgenteQlearning()  # Crea el agente de Q-Learning
tablero_prueba = np.zeros((6, 7), dtype=int)  # Genera un tablero vacio
accion_seleccionada = agente.elegir_accion(tablero_prueba)  # Selecciona una accion en el estado inicial
print(f"Acción seleccionada por el agente: {accion_seleccionada}")  # Muestra la acción elegida

Acción seleccionada por el agente: 0


In [32]:
import numpy as np

class EntrenadorCuatroEnRaya:

    def __init__(self, agente, iteraciones=50):
        self.agente = agente  # Agente de Q-Learning
        self.iteraciones = iteraciones  # Numero de partidas de entrenamiento
        self.juego = CuatroEnRaya()  # Instancia del juego

    def entrenar(self): #Empieza el entrenamiento
        for _ in range(self.iteraciones):
            self.juego.reiniciar()  # Restablece el tablero al inicio de cada partida
            estado = self.agente.obtener_estado(self.juego.tablero)  # Captura el estado inicial del tablero
            jugador = 1  # El jugador inicial siempre es 1

            while True:
                accion = self.agente.elegir_accion(self.juego.tablero)  # Agente elige una accion
                if self.juego.realizar_movimiento(accion, jugador):  # Si el movimiento es valido
                    nuevo_estado = self.agente.obtener_estado(self.juego.tablero)  # Estado despues de la jugada

                    if self.juego.verificar_victoria(jugador):  # Si el jugador gana, se asigna recompensa alta
                        recompensa = 10  # Gran recompensa por ganarxd
                        self.agente.actualizar_q(estado, accion, recompensa, nuevo_estado)
                        break

                    recompensa = -0.1  # Penalizacion
                    self.agente.actualizar_q(estado, accion, recompensa, nuevo_estado)

                    estado = nuevo_estado  # Actualiza el estado del tablero
                    jugador = 3 - jugador  # Alterna entre jugador 1 y 2

# Iniciar entrenamiento del agente
agente = AgenteQlearning()
entrenador = EntrenadorCuatroEnRaya(agente)
entrenador.entrenar()

print("¡Entrenamiento completado!")

¡Entrenamiento completado!


In [35]:
import numpy as np

class EvaluadorAgente:

    def __init__(self, agente, partidas=100):
        self.agente = agente
        self.partidas = partidas
        self.juego = CuatroEnRaya()
        self.resultados = {"victorias_agente": 0, "victorias_del_random": 0, "empates": 0}

    def jugar_partida(self):
        # Simula una partida entre el agente y un jugador aleatorio
        self.juego.reiniciar()
        estado = self.agente.obtener_estado(self.juego.tablero)
        jugador = 1  # El jugador 1 siempre empieza

        while True:
            if jugador == 1:  # Turno del agente
                accion = self.agente.elegir_accion(self.juego.tablero)
            else:  # Turno del jugador aleatorio
                accion = np.random.choice(range(self.juego.columnas))

            if self.juego.realizar_movimiento(accion, jugador):
                if self.juego.verificar_victoria(jugador):
                    self.resultados["victorias_agente" if jugador == 1 else "victorias_del_random"] += 1
                    break

                if np.all(self.juego.tablero != 0):  # Empate
                    self.resultados["empates"] += 1
                    break

                jugador = 3 - jugador  # Alterna entre jugador 1 y 2

    def evaluar(self):
        for _ in range(self.partidas):
            self.jugar_partida()

        print(f"Resultados después de {self.partidas} partidas:")
        print(self.resultados)

evaluador = EvaluadorAgente(agente)
evaluador.evaluar()

Resultados después de 100 partidas:
{'victorias_agente': 62, 'victorias_del_random': 37, 'empates': 1}
